In [1]:
#!/usr/bin/python
import cv2, os
import numpy as np
from PIL import Image
import csv
import pandas as pd
import pdb as pdb
from sklearn.cross_validation import train_test_split
import time


# Different recognizers
recognizer_LBPH = cv2.face.createLBPHFaceRecognizer()
recognizer_Fisher = cv2.face.createFisherFaceRecognizer()
recognizer_Eigen = cv2.face.createEigenFaceRecognizer()

# haar cascades for recognizing different angles
cascade_paths = ['haarcascade_frontalface_alt.xml', 'haarcascade_frontalface_default.xml', 'haarcascade_profileface.xml']

face_alt = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
face_default = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
face_profile = cv2.CascadeClassifier("haarcascade_profileface.xml")

settings = {
    'minNeighbors': 2, 
    'minSize': (40,40)
}


# train = pd.read_csv('driver_imgs_list.csv')
# mask = np.random.choice([False, True], len(train), p=[0.75, 0.25])
# train = train[mask]
# #train = train[0:500]
image_list = pd.read_csv('driver_imgs_list.csv')
train_subject_subset = image_list.subject.value_counts()[-9:].index.values
test_subject_subset = image_list.subject.value_counts()[:-9].index.values
train = image_list[image_list.subject.isin(train_subject_subset)].reset_index(drop=True)
test = image_list[image_list.subject.isin(test_subject_subset)].reset_index(drop=True)

mask = np.random.choice([False, True], len(train), p=[0.75, 0.25])
train = train[mask]
mask = np.random.choice([False, True], len(test), p=[0.75, 0.25])
test = test[mask]
print 'done getting train/test sets '


done getting train/test sets 


/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print 'getting images from paths'

getting images from paths


In [9]:
# predict_images = []
# t0 = time.time()
# scaled_size = (200,150)
# for i, image in train.iterrows():
#     image_path = './train/' + image.classname + '/' + image.img
#     predict_image_pil = Image.open(image_path).convert('L') #greyscale
#     #predict_image_pil.thumbnail(scaled_size, Image.ANTIALIAS) #resize
#     predict_image = np.array(predict_image_pil, 'uint8') #to array
#     predict_images.append(predict_image)
    
# images, predict_images, labels, predict_labels = train_test_split( predict_images, train['classname'], test_size=0.2, random_state=42)

# print("done in {}s".format(time.time() - t0))

images = []
predict_images = []
labels = [] 
predict_labels = []

t0 = time.time()
scaled_size = (200,150)

for i, image in train.iterrows():
    image_path = './train/' + image.classname + '/' + image.img
    predict_image_pil = Image.open(image_path).convert('L') #greyscale
    #predict_image_pil.thumbnail(scaled_size, Image.ANTIALIAS) #resize
    image = np.array(predict_image_pil, 'uint8') #to array
    images.append(image)
    labels.append(train['classname'][i])
    
for i, image in test.iterrows():
    image_path = './train/' + image.classname + '/' + image.img
    predict_image_pil = Image.open(image_path).convert('L') #greyscale
    #predict_image_pil.thumbnail(scaled_size, Image.ANTIALIAS) #resize
    predict_image = np.array(predict_image_pil, 'uint8') #to array
    predict_images.append(predict_image)
    predict_labels.append(test['classname'][i])
    
#images, predict_images, labels, predict_labels = train_test_split( predict_images, train['classname'], test_size=0.2, random_state=42)

print("done in {}s".format(time.time() - t0))





done in 62.6993808746s


In [10]:
print 'detecting faces from training data'

detecting faces from training data


In [11]:

# Extract relevant data
training_data = images
training_labels = labels
prediction_data = predict_images
prediction_labels = predict_labels

training_labels = map(lambda each:int(each.strip("c")), training_labels)
prediction_labels = map(lambda each:int(each.strip("c")), prediction_labels)

train_X=[]
train_Y=[]
test_X=[]
test_Y=[]

num_detect = 0
num_not_detect = 0

t0 = time.time()
for j, image in enumerate(training_data):

    #detect using different classifiers
    face = face_alt.detectMultiScale(image, **settings)
    face2 = face_default.detectMultiScale(image, **settings)
    face3 = face_profile.detectMultiScale(image, **settings)

    #Go over detected faces, stop at first detected face, return empty if no face.
    if len(face) == 1:
        facefeatures = face
    elif len(face2) == 1:
        facefeatures = face2
    elif len(face3) == 1:
        facefeatures = face3
    else:
        facefeatures = ""
        num_not_detect = num_not_detect + 1

    for (x, y, w, h) in facefeatures:
        num_detect = num_detect + 1
        train_X.append(image[y: y + h, x: x + w])
        train_Y.append(training_labels[j])
        cv2.imshow("Adding faces to traning set...", image[y: y + h, x: x + w])
        #cv2.waitKey(100)

percentage = num_detect/float((num_detect + num_not_detect))
print 'detected {} faces, out of {} ({:0.2f})'.format(num_detect, num_detect + num_not_detect, percentage)
print("done in {}s".format(time.time() - t0))



detected 761 faces, out of 1535 (0.50)
done in 171.261979103s


In [12]:
print 'predicting using LBPH'

predicting using LBPH


In [13]:

# t0 = time.time()
# correct = 0
# incorrect = 0 
# recognizer_LBPH.train(train_X,np.array(train_Y))

# for i, image in enumerate(prediction_data):

#     pred, conf = recognizer_LBPH.predict(image)

#     if pred == prediction_labels[i]:
#         correct += 1
#     else:
#         incorrect += 1
# print 'accuracy using LBPH: {}%'.format((100*correct)/(correct + incorrect))
# print("done in {}s".format(time.time() - t0))


accuracy using LBPH: 25%
done in 340.987469912s


In [14]:
print 'predicting using Fisher'

predicting using Fisher


In [21]:

t0 = time.time()
correct = 0
incorrect = 0 
recognizer_Fisher.train(train_X,np.array(train_Y))



In [22]:
for j, image in enumerate(prediction_data):

    #detect using different classifiers
    face = face_alt.detectMultiScale(image, **settings)
    face2 = face_default.detectMultiScale(image, **settings)
    face3 = face_profile.detectMultiScale(image, **settings)

    #Go over detected faces, stop at first detected face, return empty if no face.
    if len(face) == 1:
        facefeatures = face
    elif len(face2) == 1:
        facefeatures = face2
    elif len(face3) == 1:
        facefeatures = face3
    else:
        facefeatures = ""
        num_not_detect = num_not_detect + 1

    for (x, y, w, h) in facefeatures:
        num_detect = num_detect + 1
        test_X.append(image[y: y + h, x: x + w])
        test_Y.append(prediction_labels[j])
        cv2.imshow("Adding faces to testing set...", image[y: y + h, x: x + w])
        #cv2.waitKey(100)

In [ ]:
for i, image in enumerate(test_X):
    pred, conf = recognizer_Fisher.predict(image)

    if pred == test_Y[i]:
        correct += 1
    else:
        incorrect += 1
print 'accuracy using Fisher: {}%'.format((100*correct)/(correct + incorrect))
print("done in {}s".format(time.time() - t0))



In [18]:
print 'predicting using Eigen'

predicting using Eigen


In [19]:


# t0 = time.time()
# correct = 0
# incorrect = 0 
# recognizer_Eigen.train(train_X,np.array(train_Y))
# for i, image in enumerate(prediction_data):

#     pred, conf = recognizer_Eigen.predict(image)

#     if pred == prediction_labels[i]:
#         correct += 1
#     else:
#         incorrect += 1
# print 'accuracy using Eigen: {}%'.format((100*correct)/(correct + incorrect))
# print("done in {}s".format(time.time() - t0))



accuracy using Eigen: 22%
done in 2595.12101412s
